In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
import cmath #for complex numbers
from scipy.integrate import quad #for numerical integration
from sklearn.preprocessing import MinMaxScaler
import scipy

In [47]:
num_input_parameters = 6
num_output_parameters = 1
learning_rate = 0.001
num_steps = 50
batch_size = 5
num_neurons = 6

#initial values
V0 = 0.2

K = [0.4,0.5,0.6,0.7] #strikes
T = [5,6,7,8,9] #maturities

#bounds for a,b,c
bounds = np.array([[1,3],[0.1,0.6],[0,0.1]])

scaler = MinMaxScaler()

X = tf.placeholder(tf.float32, [None, num_input_parameters])
y = tf.placeholder(tf.float32, [None, num_output_parameters])

In [49]:
def brownian_motion(n, T, dim):
    dt = T/n
    r = norm.rvs(size=(dim,n+1) , scale=sqrt(dt))
    out = np.zeros((dim,n+1))        
    np.cumsum(r, axis=-1, out=out)
 
    return out

In [50]:
def euler_maruyama(mu,sigma,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + np.multiply(mu(Y[l,i],l,i),dt) + sigma(Y[l,i],l,i)*sqrt_dt*(W[l,i+1]-W[l,i])
    
    return Y

In [60]:
def BS_Call_Price(sigma, S0, K, T):
    d1 = np.log(S0/K)/np.sqrt(sigma)/T
    d2 = np.sqrt(sigma)*T/2
    dplus = d1 + d2
    dminus = d1 - d2
    return S0 * norm.cdf(dplus) + K * norm.cdf(dminus)

In [51]:
def heston_vol(a,b,c,T,W,V0):
    #assert(2*a*b > c*c) 
    
    def mu(V,i,k):
        return np.multiply(a,(b-V))
    
    def sigma(V,i,k):
        return np.multiply(c,np.sqrt(np.maximum(0.0,V)))
    
    return euler_maruyama(mu,sigma,T,V0,W)

In [52]:
def next_batch_heston_vol_EM_train(batch_size,bounds):
    X = np.zeros((batch_size,num_input_parameters))
    y = np.zeros((batch_size,num_output_parameters))

    X[:,0] = uniform.rvs(size=batch_size)*(bounds[0][1]-bounds[0][0]) + bounds[0][0]#a
    X[:,1] = uniform.rvs(size=batch_size)*(bounds[1][1]-bounds[1][0]) + bounds[1][0] #b
    X[:,2] = uniform.rvs(size=batch_size)*(bounds[2][1]-bounds[2][0]) + bounds[2][0] #c

    n = 100
    dim = 10

    for i in range(batch_size):
        for j in range(len(T)):
            W = brownian_motion(n, T[j], dim)
            for k in range(len(K)):
                y[i,0] = np.mean(heston_vol(X[i,0],X[i,1],X[i,2],T[j],W,V0)[:,n])

    return X,y

In [53]:
#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
hidden4 = fully_connected(hidden3, num_neurons, activation_fn=tf.nn.elu)

outputs = fully_connected(hidden4, num_output_parameters, activation_fn=None)

In [54]:
#Loss Function
loss = tf.reduce_mean(tf.sqrt(tf.square(outputs - y)))  # MSE

In [55]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [56]:
init = tf.global_variables_initializer()

In [57]:
saver = tf.train.Saver()

In [58]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch_heston_vol_EM_train(batch_size,bounds)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 10 == 0:
            
            rmse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tRMSE:", rmse)
    
    saver.save(sess, "./models/heston_vol_nn")

0 	RMSE: 3.3891582
10 	RMSE: 0.094429046
20 	RMSE: 0.07287586
30 	RMSE: 0.24877004
40 	RMSE: 0.16718525
